## Name Binding (What You Think of as Assignment)
Whenever you use assignment (=), you can think about it as putting a name tag (named with the variable name) onto the data--this is called name binding. Yoy can use the `id()` function to see the object's memory address. Hence, for the duration of the object's existence (ie not deleted), then the id will be unique and unchanged. Python's assigment (=) NEVER copies data--it merely puts another name tag on the same object. The `is` keyword in Python is highly related to `id()` in that `is` detects whether the identity of 2 objects are the same--if the 2 variables are pointing to same memory address. If `x is y` is True, then `id(x) == id(y)`.

If you want to see how many name tags are attached to the same object, then use `sys.getrefcout`. Python internally has a reference count for every object--once the reference count goes to 0, then the object is deleted from memory.  
Notice this caveat from the documentation:  
```The count returned is generally one higher than you might expect, because it includes the (temporary) reference as an argument to getrefcount().```  
Deleting a variable will remove the name tag. Python's `del` statement doesn't directly delete a variable. Instead `del` deletes the name binding (the name tag) onto the object. Once an object's reference count goes to 0, then Python's garbage collector will finally remove the object from memory.

Python has data structures that are mutable (list, set, dictionary) and some are immutable (tuple, string, numeric [float, int, bool]). 

In [1]:
import sys

a = []
print("reference count: {}; memory location: {}".format(sys.getrefcount(a), id(a)))

reference count: 2; memory location: 1836773559496


In [2]:
# a list is a mutable object
a = []
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))
print(a is b)
print(id(a), id(b))
a.append(None)
print(a, b)

2
3
True
1836773559880 1836773559880
[None] [None]


In [3]:
# an tuple is immutable--you cannot make a number into another number. That will occupy different memory locations
a = (1, )
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))
print(a is b)
print(id(a), id(b))
a += (2,)
print(a, b)
print(a is b)
print(id(a), id(b)) # notice that id(a) has changed memory location
print(sys.getrefcount(a)) # notice that the reference count has fell back to 2

2
3
True
1836774045960 1836774045960
(1, 2) (1,)
False
1836756384200 1836774045960
2


R is pass by value--that means when you pass a dataframe into a function, the dataframe is copied. If you mutate the dataframe inside an R function, nothing will happen to the original dataframe.  
This is not true in Python.  In contrast, Python's assigment (=) NEVER copies data, not even in a function's argument assignment. Hence, when you pass a variable into the function, that variable can be mutated if it is a mutable type. Python is not pass by value nor pass by reference. Instead Python is pass by object. 

In [4]:
# mutable object changed
def appender(lst):
    lst.append(None)
    
a = []
print(a)
appender(a)
print(a)

[]
[None]


In [5]:
# immutable object remains unchanged
def concatenate(string):
    string = "Hello " + string # this new `string` variable is no longer bound to the same object. 
    print(id(string), string) # You have moved the "name tag" to a different object.

string = "ML Study Group"
print(id(string), string)
concatenate(string)
print(id(string), string)

1836774329904 ML Study Group
1836774342352 Hello ML Study Group
1836774329904 ML Study Group


If fact, you change to multiple assignment. The right-most expression is evaluated first and then bound to the variables left to right.

In [6]:
a = b = 9000 # bind 900 to `a` first, then `b`
print(a, b)
print(a is b)

9000 9000
True


In [7]:
a, b = 1, 2 # tuple unpacking
a, b = b, a # this is a safe operation in Python. You don't need a temporary third variable.
print(a, b)

2 1


In [8]:
# If you combine tuple unpacking and multiple assignment, you get this is fun puzzle. 
# The values for a and b are swapped and then swapped back. Basically, nothing happens.
a, b = 1, 2

(b, a) = (a, b) = (a, b)
a, b

(1, 2)

To prove that multiple assigment happens left to right, so I use a `property` attribute. I'll explain `property` later.

In [9]:
class SillyAssignment:
    def __init__(self, value):
        self.id = value
        self._gotit = None
        self._counter = 0
    
    @property
    def gotit(self):
        print("Getter {}: {}".format(self.id, self._gotit))
        return self._gotit
        
    @gotit.setter
    def gotit(self, value):
        self._counter += 1
        print("Setter {} to {}; set {} time(s)".format(self.id, value, self._counter))
        self._gotit = value

In [10]:
a, b = 1, 2

s1 = SillyAssignment("s1")
s2 = SillyAssignment("s2")

In [11]:
s2.gotit, s1.gotit = s1.gotit, s2.gotit = (a, b)

Setter s2 to 1; set 1 time(s)
Setter s1 to 2; set 1 time(s)
Setter s1 to 1; set 2 time(s)
Setter s2 to 2; set 2 time(s)


## Variable Scoping
How do you know which `variable` you get when variables have the same name? Python has resolves the conflict with scope resolution following the LEGB rule: Local, Enclosing, Global, Built-in. If a variable is not found in the local scope, go to the outer scopes. If not found at all, then raise `NameError` exception. Variables with the same name in the same scope will overwrite each other--only 1 can exist at a time.
<p align="center"><img src="images/LEGB_Variable_Scoping.png"></p>
`Local`: inside a function, look to see if variable is defined. If it is there, use that one.

In [9]:
a = 1 # this is a global variable
def a(): # notice even the function name is the same as the others
    a = 2 # this is the local variable
    return a
get_a()

2

`Enclosing`: if not local variable found inside nested function, look for the variable in the outer function.

In [22]:
a = 1 # global variable will be overwritten by the following function
def a(): # this function overwrites/replaces the previous assignment. The value of 1 is totally lost/gone.
    a = 2 # get this enclosing variable
    def b():
        return a
    return b()
a()

2

`Global`: variables defined at the module level. In your function call, if variables are not defined locally or enclosing, go to the module level to find the variable.

In [27]:
a = 1 # global variable
def func1():
    def func2():
        return a
    return func2()
func1()

1

`Global` variables are not _truly_ global in the sense that variables cannot communicate between modules. If `pandas` and `sklearn` both have a variable `a`, then they cannot communicate with each other. `pandas.a` does not know about `sklearn.a`.

In [30]:
%%writefile temp1.py
a = 1

Writing temp1.py


In [31]:
%%writefile temp2.py
a = 2

Writing temp2.py


In [32]:
%%writefile temp3.py
print(a)

Writing temp3.py


In [33]:
import temp1
print(temp1.a)

1


In [34]:
import temp2
print(temp2.a)

2


In [35]:
import temp3 # fails to load

NameError: name 'a' is not defined

In [36]:
a = 1 # even if you define it now, temp3 cannot be imported since `a` in temp3 does not exist
import temp3

NameError: name 'a' is not defined

As a general rule, inner scopes cannot assign outer scoped variables, so they can mutate them. For example, lists are mutatable.

In [40]:
a = []
def inner():
    a.append(None)
print(a)
inner()
print(a) # a is mutated

[]
[None]


In [42]:
a = []
def inner():
    a = []
    a.append(None)
print(a)
inner()
print(a) # a is not mutated or assigned. Basically nothing has happened

[]
[]


There is an exception to this rule by using `global` or `nonlocal` (which are the only declarative statements that Python has). `global` tells Python you are using the global

In [44]:
a = 1
def reassign():
    global a
    a = 2

print(a)
reassign()
print(a)

1
2


In [55]:
a = 1
def reassign():
    a = 2
    def func():
        global a
        print(a, "I am inside nested function")
        a = 3
    func()
print(a)
reassign()
print(a)

1
1 I am inside nested function
3


In [61]:
def global_assign():
    global xyz
    xyz = 1
    
try:
    del xyz
    print("deleted xyz")
except NameError:
    print("xyz doesn't exist")

try:
    print(xyz)
except NameError:
    print("xyz doesn't exist")

global_assign()
print(xyz)

deleted xyz
xyz doesn't exist
1


In [74]:
def func1():
    a = 1
    def func2():
        nonlocal a
        a = 2
    print(a)
    func2()
    print(a)
func1()

1
2


In [ ]:
builtins can be overwritten

In [67]:
a = 1 # global variable will be overwritten by the following function
def a(): # this function overwrites/replaces the previous assignment. The value of 1 is totally lost/gone.
    def b():
        return a # refers to the outer function
    return b()
print(a())
a() is a()() is a()()()

<function a at 0x0000021A6A4C8F28>


True

In [ ]:
recursion

### Style Guide

In [1]:
# I usually use this type of syntax for prints where I put all the variables at the end. This is up to your personal preference
first_name = 'Peter'
last_name = 'Pan'
print("Hello, my name is {} {}".format(first_name, last_name))
# You can change the order using named arguments and also repeat the arguments
print("Last name: {last_name}; first name: {first_name}. Again my name is {first_name} {last_name}".format(first_name=first_name, last_name=last_name))

# f**** it! You can use the new f-string syntax that came out in Python 3.6
print(f"Hello, my name is {first_name} {last_name}")
# have you ever considered nested string formatting? ;-)
print(f"{f'I am {42 -29} years old!'}")

Hello, my name is Peter Pan
Last name: Pan; first name: Peter. Again my name is Peter Pan
Hello, my name is Peter Pan
I am 13 years old!


In [ ]:
# Context manager
# If you like, use this syntax since it guarantees that the file will be closed after everything in the code block is run.
# It just saves one line (don't have to close file manually) and also make the code block for what you want to do
# the file very obvious due to the indentation.
with open('file_here') as f:
    f.read()

In [2]:
# Instead of \ to denote line continuation, I use parenthesis--also works for brackets [].
# Good for function calls with lots of arguments
sum([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
     6, 7, 8, 9, 0])
print("This is my super looooooooooooooooooooooooooooooong string "
     "and it ends here!")

# Implicit string concatenation: notice that string literals are automatically appended to each other without '+' operator
print("asdf" "asdf" == 'asdfasdf')
# Hence, parenthesis and long strings work well together.
my_long_string = ('This is my super long string that never ends because I do not want to stop '
                  "typing for some reason until I'm out of breath!")
my_long_string_2 = ('I like to count to big numbers. I start at {} '
                    'and finally end up at {}'
                    .format(1, int(1e10)))
print(my_long_string)
print(my_long_string_2)

This is my super looooooooooooooooooooooooooooooong string and it ends here!
True
This is my super long string that never ends because I do not want to stop typing for some reason until I'm out of breath!
I like to count to big numbers. I start at 1 and finally end up at 10000000000


In [ ]:
# PEP8 is a nice style guide for readability. I try my best, but even I don't get it right every time.
# http://pymbook.readthedocs.io/en/latest/pep8.html
# If you are very fancy, you can have Python automatically format your code to conform to pep8 if you type this in the Terminal
autopep8 your_python_script_here.py
# This will print to Terminal the correctly formatted script but doesn't save it. 
# If you want to save the results back into your script, you can use the argument --in-place
autopep8 --in-place your_python_script_here.py

In [3]:
# I usually use triple hash sign ### when I need to make an important comment. Usually to mention something is hard coded.
# This is not a Python PEP8 style. It's just a personal preference to make note that this is not a regular comment.
PI = 3.14159265 ### this is hard coded

As a personal preference, I prefer to write pure functions with no side effects where if you put in the same input, you always get the same output. When practical, I try to avoid functions that mutate whatever is inputted. Pure functions are easier to debug. Of course, sometimes it's just much easier or runs faster with mutation. Then I add a triple hash sign to say that this function mutates the underlying object.

In [1]:
class Issue():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
bbw = Issue(title="Bloomberg", price=5.99, pages=112)

TODOS:  
eval/exec, ast.eval  
multiple assignment; multiple comparison x < y < z  

if/and lazy evaulation short circuit  
new features: ordered dictionary,walrus operator  type annotations/static typing  